In [1]:
import json

In [2]:
import time

In [3]:
from tqdm import tqdm

In [4]:
with open('grade_school_math/data/test.jsonl', 'r') as file_split:
    json_list = list(file_split)

split = []
for json_str in json_list:
    result = json.loads(json_str)
    split.append(result)

In [8]:
len(split)

1319

In [7]:
split[0]

{'question': "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?",
 'answer': 'Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.\nShe makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.\n#### 18'}

In [5]:
import re
import ast

In [10]:
import amrlib
import spacy
amrlib.setup_spacy_extension()
nlp = spacy.load('en_core_web_sm')

/data/wang/zhenyub/condaenvs/cenv_x86/lib/python3.8/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.4.1) was trained with spaCy v3.4.0 and may not be 100% compatible with the current version (3.7.6). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, LogitsProcessor, LogitsProcessorList
from functools import reduce

/data/wang/zhenyub/condaenvs/cenv_x86/lib/python3.8/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [7]:
import transformers
access_token = "KEY"

In [8]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct", token=access_token)
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Meta-Llama-3.1-8B-Instruct", torch_dtype=torch.float16, low_cpu_mem_usage=True, device_map="auto", token=access_token
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [11]:
from groq import Groq

In [19]:
client = Groq(
    api_key='KEY',
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama-3.1-70b-versatile",
)

print(chat_completion.choices[0].message.content)

Fast language models are crucial in today's world of rapid communication and information exchange. Their importance can be seen in several areas:

1. **Response Time**: Fast language models can promptly respond to user queries, providing immediate answers and improving the overall user experience. This is particularly important for applications such as chatbots, virtual assistants, and language translation software.

2. **Efficient Processing**: Fast language models can efficiently process large amounts of data, reducing computation time and energy consumption. This allows for more efficient deployment on devices with limited processing power, such as mobile devices or smart home devices.

3. **Real-Time Analysis**: Fast language models enable real-time analysis of text data, such as sentiment analysis, topic modeling, and entity recognition. This is crucial for applications like social media monitoring, customer feedback analysis, and news sentiment analysis.

4. **Large-Scale Applica

In [12]:
messages = [
    {"role": "system", "content": "You are a helpful and honest assistant. \
      Respond concisely and truthfully."},
    {"role": "user", "content": "Explain Einstein's relativity theory in three sentences."},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.5,
    top_p=0.9,
    num_return_sequences=2,
)
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))
response1 = outputs[1][input_ids.shape[-1]:]
print(tokenizer.decode(response1, skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Albert Einstein's theory of relativity is composed of two main components: special relativity and general relativity. Special relativity, introduced in 1905, states that the laws of physics are the same for all observers in uniform motion and that the speed of light is always constant, regardless of the observer's frame of reference. General relativity, introduced in 1915, expands on this idea by describing gravity as the curvature of spacetime caused by massive objects, which affects not only objects with mass but also the passage of time itself.
Albert Einstein's theory of relativity consists of two main components: special relativity and general relativity. Special relativity, introduced in 1905, posits that the laws of physics are the same for all observers in uniform motion relative to one another, and that the speed of light is always constant, regardless of the observer's frame of reference. General relativity, introduced in 1915, expands on this idea by describing gravity as th

In [17]:
def Answer_Agent_IO(question='', num_seq=1):
    message = \
    f"""Given a question, give out only the direct answer, without any explanations. \
    Make sure to include your answer in the format: ###answer###. \
    Here are two examples as guideline: \
    "Question: John bought 3 cars, each worth $30000. How much did John pay in total for his 3 cars? \
    Answer: ###$9000###. \
    Question: Payton has 9 apples that she is planning to evenly split to 3 of her friends. How many apples would each of her friend get? \
    Answer: ###3###." \
    Make sure to include your answer in the format: ###answer###. \
    Question: {question} \
    Answer: \
    """
    
    messages_in = [{"role": "system", "content": \
                     "You are a faithful agent that answer mathematical questions accurately."},\
                {"role": "user", "content": message}]
    
    chat_completion = client.chat.completions.create(
        messages=messages_in,
        model="llama-3.1-70b-versatile",
    )

    # print(chat_completion.choices[0].message.content)

    # responses = []
    # n = 0
    # while n < num_seq:
    #     responses.append(tokenizer.decode(outputs[n][input_ids.shape[-1]:], skip_special_tokens=True))
    #     n += 1
    
    return chat_completion.choices[0].message.content

In [24]:
def Answer_Agent_CoT_2Shot(question='', num_seq=1):
    message = \
    f"""Given a question, give out your chain of thought process and the final answer. \
    Make sure to include your answer in the format: ###answer###. \
    Here are two examples as guideline: \
    "Question: John bought 3 cars, each worth $30000. He also bought 3 bikes, each worth $1000. How much did John pay in total? \
    Thought: John paid 3 * $30000 = $90000 for the cars, and 3 *$ 1000 = $3000 for the bikes. The total amounts to $90000 + $3000 = $93000. \
    Answer: ###$93000###. \
    Question: Each day, Payton brings 9 apples that she would split to 3 of her friends. In 3 days, how many apples would each of her friend get in total? \
    Thought: Each of the 3 friends would get 9 / 3 = 3 apples per day. For 3 days, each of her friend would get 3 * 3 = 9 apples in total. \
    Answer: ###9###." \
    Make sure to include your final answer in the format: ###answer###. \
    Question: {question} \
    Answer: \
    """
    
    messages_in = [{"role": "system", "content": \
                     "You are a faithful agent that answer mathematical questions accurately."},\
                {"role": "user", "content": message}]
    
    chat_completion = client.chat.completions.create(
        messages=messages_in,
        model="llama-3.1-70b-versatile",
    )

    # print(chat_completion.choices[0].message.content)

    # responses = []
    # n = 0
    # while n < num_seq:
    #     responses.append(tokenizer.decode(outputs[n][input_ids.shape[-1]:], skip_special_tokens=True))
    #     n += 1
    
    return chat_completion.choices[0].message.content

In [25]:
def Answer_Agent_CoT_0Shot(question='', num_seq=1):
    message = \
    f"""Given a question, give out your chain of thought process and the final answer. \
    Make sure to include your answer in the format: ###your answer###. \
    Question: {question} \
    Let's think step by step. \
    Answer: \
    """
    
    messages_in = [{"role": "system", "content": \
                     "You are a faithful agent that answer mathematical questions accurately."},\
                {"role": "user", "content": message}]
    
    chat_completion = client.chat.completions.create(
        messages=messages_in,
        model="llama-3.1-70b-versatile",
    )

    # print(chat_completion.choices[0].message.content)

    # responses = []
    # n = 0
    # while n < num_seq:
    #     responses.append(tokenizer.decode(outputs[n][input_ids.shape[-1]:], skip_special_tokens=True))
    #     n += 1
    
    return chat_completion.choices[0].message.content

In [22]:
s = '###123###'
ast.literal_eval(re.search('###(.*)###', s).group(1))

123

In [23]:
all_answer_lm3_text_io = []
ori_answers_io = []
all_counts = -1
for s in tqdm(split[0:100]):
    all_counts += 1
    ori_question = s['question']
    
    response = Answer_Agent_IO(ori_question, 1)
    ori_answers_io.append(response)
    try:
        response_format = re.search('###(.*)###', response).group(1)
        try:
            answer_current = ast.literal_eval(response_format)
        except:
            answer_current = response_format
        all_answer_lm3_text_io.append(answer_current)
    except:
        try:
            answer_current = ast.literal_eval(response)
            all_answer_lm3_text_io.append(answer_current)
        except:
            try:
                response_format = response.split('Answer: ')[1]
                answer_current = ast.literal_eval(response_format)
                all_answer_lm3_text_io.append(answer_current)
            except:
                print('no answer provided')
                print(response)
                all_answer_lm3_text_io.append('')
                continue

100%|██████████| 100/100 [03:32<00:00,  2.13s/it]


In [26]:
all_answer_lm3_text_cot = []
all_counts = -1
for s in tqdm(split[0:100]):
    all_counts += 1
    ori_question = s['question']
    
    response = Answer_Agent_CoT_2Shot(ori_question, 1)
    try:
        response_format = re.search('###(.*)###', response).group(1)
        try:
            answer_current = ast.literal_eval(response_format)
        except:
            answer_current = response_format
        all_answer_lm3_text_cot.append(answer_current)
    except:
        try:
            answer_current = ast.literal_eval(response)
            all_answer_lm3_text_cot.append(answer_current)
        except:
            try:
                response_format = response.split('Answer: ')[1]
                answer_current = ast.literal_eval(response_format)
                all_answer_lm3_text_cot.append(answer_current)
            except:
                print('no answer provided')
                print(response)
                all_answer_lm3_text_cot.append('')
                continue

100%|██████████| 100/100 [08:50<00:00,  5.30s/it]


In [27]:
all_answer_lm3_text_cot_0s = []
all_counts = -1
for s in tqdm(split[0:100]):
    all_counts += 1
    ori_question = s['question']
    
    response = Answer_Agent_CoT_0Shot(ori_question, 1)
    try:
        response_format = re.search('###(.*)###', response).group(1)
        try:
            answer_current = ast.literal_eval(response_format)
        except:
            answer_current = response_format
        all_answer_lm3_text_cot_0s.append(answer_current)
    except:
        try:
            answer_current = ast.literal_eval(response)
            all_answer_lm3_text_cot_0s.append(answer_current)
        except:
            try:
                response_format = response.split('Answer: ')[1]
                answer_current = ast.literal_eval(response_format)
                all_answer_lm3_text_cot_0s.append(answer_current)
            except:
                print('no answer provided')
                print(response)
                all_answer_lm3_text_cot_0s.append('')
                continue

  0%|          | 0/100 [00:00<?, ?it/s]

 29%|██▉       | 29/100 [02:03<04:32,  3.83s/it]

no answer provided
To find out the distance between Henry's first and second stops, let's think step by step.

Step 1: First, let's find out the total distance of the trip. The problem already tells us this information: the trip is 60 miles.

Step 2: Then, we'll find out how far Henry had traveled when he made each of his stops. For the first stop, he traveled 20 miles. 

Step 3: For the second stop, we are told it was 15 miles before the end. So we must find out how far from the start the end is - that is 60 miles - then subtract 15. This is written as 60 - 15 = 45. Henry had traveled 45 miles before he made his second stop. 

Step 4: Now we know the mileage for both the first stop and the second stop. If we subtract the distance traveled at the first stop (20 miles) by the distance traveled at the second stop (45 miles), this will tell us the mileage between the two stops. This is 45 - 20 = 25. 

The answer is: 25 miles


 71%|███████   | 71/100 [04:54<01:54,  3.93s/it]

no answer provided
To solve this problem, I'd follow these steps:

1. **Calculate the total number of classes on weekdays:** Judy teaches 5 dance classes per day, and there are 5 weekdays (Monday to Friday), so the total number of classes on weekdays is 5 classes/day * 5 days = 25 classes.

2. **Calculate the total number of classes on Saturday:** Judy teaches 8 dance classes on Saturday.

3. **Calculate the total number of classes in a week:** Add the total number of classes on weekdays and Saturday, 25 classes + 8 classes = 33 classes.

4. **Calculate the total number of students per week:** Each class has 15 students, and there are 33 classes in a week, so the total number of students per week is 15 students/class * 33 classes = 495 students.

5. **Calculate the total amount of money earned per week:** Judy charges $15.00 per student, and there are 495 students, so the total amount of money earned per week is $15.00/student * 495 students = $7425.

**So, the amount of money Judy mak

100%|██████████| 100/100 [06:54<00:00,  4.15s/it]


In [51]:
all_answer_lm3_text_io_ensemble3 = []
all_counts = -1
for s in tqdm(split[0:100]):
    all_counts += 1
    ori_question = s['question']
    
    responses = Answer_Agent_IO(ori_question, 3)
    answer_current = []

    for response in responses:
        try:
            response_format = re.search('###(.*)###', response).group(1)
            try:
                answer_current.append(ast.literal_eval(response_format))
            except:
                answer_current.append(response_format)
        except:
            try:
                answer_current.append(ast.literal_eval(response))
            except:
                try:
                    response_format = response.split('Answer: ')[1]
                    answer_current.append(ast.literal_eval(response_format))
                except:
                    print('no answer provided')
                    print(response)
                    answer_current.append('')
                    continue
    all_answer_lm3_text_io_ensemble3.append(answer_current)

100%|██████████| 100/100 [04:49<00:00,  2.89s/it]


In [52]:
all_answer_lm3_text_io_ensemble3

[['$38', '$34', '$34'],
 [4, 4, 4],
 ['$106,000', '$98,000', '$110,000'],
 [1080, 1080, 1080],
 [65, 65, 75],
 ['$80', '$96', '$80'],
 [180, 180, 140],
 ['(200 * 0.4 * 2) + (200 * 0.6 * 2) + 20', 200, 200],
 ['distance = 0',
  '###distance = 0###',
  'John drives 3 hours at 60 mph, so he covers 3 * 60 = 180 miles from home initially. Then, he drives 0.5 hours at 30 mph, so he covers 0.5 * 30 = 15 miles. The remaining distance to home is (4 - 0.5 - 2) * 80 = 1.6 * 80 = 128 miles. So the total distance is 180 + 15 + 128 = 323 miles.'],
 ['$460', '$460', '$450 + ($10 * 5 * 1.2)'],
 [180, 240, 180],
 ['3 * $68 = $204', '3 * $68 = $204', '3 * $68 = $204'],
 [7, 7, 7],
 [12, 12, 12],
 ['60%', '60%', '60%'],
 ['$150', '$500', '$-500'],
 ["Since the question doesn't specify the distance covered by each train, we'll assume they cover the same distance. So, each train would cover 80 + 150 = 230 miles in the two days.",
  'Since the trains travel the same distance, we can find the total distance 

In [28]:
import sys
import re
import string
from collections import Counter
import pickle

In [29]:
def normalize_answer(s):

    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

In [37]:
with open('answers_lm3_io_70b.txt', 'w') as f:
    f.write('\n'.join(str(ans) for ans in all_answer_lm3_text_io))

In [31]:
with open('answers_lm3_io_ori_70b.txt', 'w') as f:
    f.write('\n'.join(str(ans) for ans in ori_answers_io))

In [32]:
with open('answers_lm3_cot_0Shot_70b.txt', 'w') as f:
    f.write('\n'.join(str(ans) for ans in all_answer_lm3_text_cot_0s))

In [33]:
with open('answers_lm3_cot_2Shot_70b.txt', 'w') as f:
    f.write('\n'.join(str(ans) for ans in all_answer_lm3_text_cot))

In [43]:
with open('answers_golden.txt', 'w') as f:
    f.write('\n'.join(s['answer'].split('#### ')[1] for s in split[0:100]))

In [34]:
ANS_RE = re.compile(r"(\-?[0-9\.\,]+)")
INVALID_ANS = "[invalid]"

def extract_answer(completion):
    match = ANS_RE.search(completion)
    if match:
        match_str = match.group(1).strip()
        match_str = match_str.replace(",", "")
        return match_str
    else:
        return INVALID_ANS

In [35]:
def is_correct(model_completion, gt_example):
    gt_answer = extract_answer(gt_example)
    assert gt_answer != INVALID_ANS
    return extract_answer(model_completion) == gt_answer

In [40]:
with open('answers_golden.txt', 'r') as file:
    golden = file.readlines()

In [38]:
with open('answers_lm3_io_70b.txt', 'r') as file:
    output_io = file.readlines()

with open('answers_lm3_cot_0Shot_70b.txt', 'r') as file:
    output_cot_0Shot = file.readlines()

with open('answers_lm3_cot_2Shot_70b.txt', 'r') as file:
    output_cot_2Shot = file.readlines()

In [36]:
def evaluation(output, golden):
    correct = 0
    length = len(golden)

    for (o, g) in zip(output, golden):
        res = is_correct(o, g)
        if res:
            correct += 1
    
    return correct, correct/length

In [41]:
evaluation(output_io, golden)

(29, 0.29)

In [42]:
evaluation(output_cot_0Shot, golden)

(87, 0.87)

In [43]:
evaluation(output_cot_2Shot, golden)

(93, 0.93)

In [ ]:
def Answer_Agent_debate(question='', prior=[], others='', num_seq=1):
    message = \
    f"""Given a question, and the reasoning processes from another agent, recheck your thoughts and answer. \
    Make sure to include your answer in the format: ###answer###. \
    Question: {question} \
    Reasoning from another agent: {others} \
    Thought: \
    """
    
    messages = [{"role": "system", "content": \
                        "You are a faithful agent that answer mathematical questions accurately."}]
    for p in prior:
        messages.append({"role": "user", "content": p})
    messages.append({"role": "user", "content": message})
    
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=512,
        eos_token_id=terminators,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.5,
        top_p=0.9,
        num_return_sequences=num_seq,
    )

    responses = []
    n = 0
    while n < num_seq:
        responses.append(tokenizer.decode(outputs[n][input_ids.shape[-1]:], skip_special_tokens=True))
        n += 1
    
    return responses